In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re
import nltk
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

In [5]:
train_df = pd.read_csv("../amazon_review_full_csv/train.csv", header=None, names=["label","title","review"], 
                       on_bad_lines='skip',  # pour ignorer les lignes corrompues
                       encoding="utf-8")

test_df = pd.read_csv("../amazon_review_full_csv/test.csv", header=None, names=["label","title","review"], 
                      on_bad_lines='skip', 
                      encoding="utf-8")

In [6]:
#test affichage
train_df.head(5)


,label,title,review
0,3,more like funchuck,Gave this to my dad for a gag gift after direc...
1,5,Inspiring,I hope a lot of people hear this cd. We need m...
2,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
3,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...
4,5,Too good to be true,Probably the greatest soundtrack in history! U...


In [7]:
# Distrib des labels (pour l'instant de 1 à 5)
train_df['label'].value_counts()


label
3    600000
5    600000
4    600000
1    600000
2    600000
Name: count, dtype: int64

In [8]:
# Préparation des données
#On va regrouper les labels 1,2 ; 3 ; 4,5 en 3 classes : négatif, neutre, positif
def map_label(x):
    if x in [1,2]:
        return 1  # negatif
    elif x == 3:
        return 2  # neutre
    else:
        return 3  # positif

train_df['label_3'] = train_df['label'].apply(map_label)

train_df['label_3'].value_counts()


label_3
3    1200000
1    1200000
2     600000
Name: count, dtype: int64

In [9]:
print("Valeurs nulles par colonne:")
print(train_df.isna().sum())

Valeurs nulles par colonne:
label        0
title      188
review       0
label_3      0
dtype: int64


In [10]:
train_df.head()

,label,title,review,label_3
0,3,more like funchuck,Gave this to my dad for a gag gift after direc...,2
1,5,Inspiring,I hope a lot of people hear this cd. We need m...,3
2,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...,3
3,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...,3
4,5,Too good to be true,Probably the greatest soundtrack in history! U...,3
